In [83]:
import pandas as pd

# Replace 'your_file.xlsx' with the actual path to your Excel file
finance_options = pd.read_excel("financem.xlsx")

finance_options

,Module,Year,Optional/Compulsory,Prerequisites,Units
0,MA108,1.0,0,0.0,0.5
1,FM100,1.0,0,0.0,0.5
2,FM102,1.0,0,0.0,0.5
3,EC1A3,1.0,0,0.0,0.5
4,EC1B3,1.0,0,0.0,0.5
5,ST102,1.0,0,0.0,1.0
6,AC102,1.0,1,0.0,0.5
7,ST101,1.0,1,0.0,0.5
8,FM201,2.0,0,0.0,0.5
9,FM200,2.0,0,0.0,0.5


In [291]:
import pandas as pd
import numpy as np

# Load your CSV
df = pd.read_csv('data/modules/marks_summary_modules.csv')

# Step 1: Filter out rows that are not from 2023/24 (i.e., keep only modules that appear in 2023/24 at least once)
latest_year = '2023/24'
valid_modules = df[df['year'] == latest_year]['code'].unique()
df = df[df['code'].isin(valid_modules)]

# Step 2: Define an aggregation function
def aggregate_module(group):
    total_takers = group['marks'].sum()
    
    # Avoid division by zero
    if total_takers == 0:
        return pd.Series({
            'Mean': np.nan,
            'q10': np.nan,
            'Median': np.nan,
            'q90': np.nan,
            'Pooled_SD': np.nan,
            'Average Takers Per Year': 0,
            'Years Counted': 0
        })

    # Weighted mean
    mean = np.average(group['mean'], weights=group['marks'])
    
    # Simple averages for percentiles
    q10 = group['q10'].mean()
    median = group['median'].mean()
    q90 = group['q90'].mean()
    
    # Pooled standard deviation
    numer = ((group['marks'] - 1) * group['sd'] ** 2).sum()
    denom = group['marks'].sum() - group.shape[0]
    pooled_sd = np.sqrt(numer / denom) if denom > 0 else np.nan

    return pd.Series({
        'mean': mean,
        'q10': q10,
        'Median': median,
        'q90': q90,
        'Pooled_SD': pooled_sd,
        'Total Takers': total_takers/group.shape[0],
        'Years Counted': group.shape[0]
    })

# Step 3: Group and aggregate
summary_df = df.groupby('code').apply(aggregate_module).reset_index()

summary_df

outside_options = pd.read_csv('data/modules/outside_options.csv')

# Merge with summary_df on course code
outside_options_marks = outside_options.merge(
    summary_df,
    left_on='code',
    right_on='code',
    how='left'
)
def parse_list_string(s):
    if pd.isna(s) or s.strip() == '[]':
        return []
    return [item.strip().strip("'\"") for item in s.strip('[]').split(',')]

outside_options_marks.drop(columns=['course', 'total_students', 'avg_class_size', 'capped'], inplace=True)

outside_options_marks.sort_values(by='mean', ascending=False, inplace=True)
outside_options_marks['parsed_prereqs'] = outside_options_marks['prerequisites'].apply(parse_list_string)
outside_options_marks['parsed_exclusions'] = outside_options_marks['mutually_exclusive_courses'].apply(parse_list_string)
outside_options_marks

,code,mean,q10,Median,q90,Pooled_SD,Total Takers,Years Counted
0,AC102,71.771288,52.62,74.8,87.58,14.904358,549.6,5.0
1,AC103,65.461790,47.86,64.7,79.64,12.328764,281.6,5.0
2,AC105,70.500000,59.40,71.0,82.00,9.600000,115.0,1.0
3,AC106,59.400000,47.00,60.0,70.00,10.200000,115.0,1.0
4,AC200,59.511446,48.92,60.2,70.56,9.969796,132.8,5.0
...,...,...,...,...,...,...,...,...
442,ST313,71.304167,66.10,70.5,76.10,4.915282,24.0,2.0
443,ST314,62.000000,46.80,61.5,77.80,12.800000,22.0,1.0
444,ST326,58.879808,35.22,62.3,78.02,17.857641,62.4,5.0
445,ST327,67.521402,58.38,69.3,77.14,11.250656,54.2,5.0


In [266]:
import pandas as pd
import random
from itertools import combinations as iter_combinations

# Step 1: Filter compulsory modules
compulsory_modules = finance_options[finance_options['Optional/Compulsory'] == 0]

# Step 2: Filter optional modules that are not 'OO'
optional_modules = finance_options[
    (finance_options['Optional/Compulsory'] == 1) &
    (~finance_options['Module'].str.contains('OO'))
]

# Step 3: Preload unit dictionaries
module_units_dict = finance_options.set_index('Module')['Units'].to_dict()
outside_units_dict = outside_options_marks.set_index('code')['units'].to_dict()

# Step 4: Build combinations of compulsory + one optional module
initial_combinations = []

for _, optional_row in optional_modules.iterrows():
    # Base compulsory modules
    modules_list = compulsory_modules['Module'].tolist()

    # Add optional module
    modules_list.append(optional_row['Module'])

    # Calculate total units
    total_units = sum(module_units_dict.get(mod, 0) for mod in modules_list)

    # Add the combination along with the optional module chosen
    initial_combinations.append({
        'Modules': modules_list,
        'Total Units': total_units,
        'Optional Module Chosen': optional_row['Module']
    })
    
initial_combinations_df = pd.DataFrame(initial_combinations)
initial_combinations_df2 = initial_combinations_df


In [265]:
grouped_valid_combos = []

for idx, row in initial_combinations_df.iterrows():
    initial_modules = row['Modules']
    
    def is_valid(row_oo):
        # Check prerequisites
        prereqs = row_oo['parsed_prereqs']
        if prereqs and not any(mod in initial_modules for mod in prereqs):
            return False

        # Check mutual exclusions
        exclusions = row_oo['parsed_exclusions']
        if exclusions and any(mod in initial_modules for mod in exclusions):
            return False
        
        # Ensure that no module in initial_modules is also in the valid outside option
        if row_oo['code'] in initial_modules:
            return False
        
        return True

    # Apply filters
    valid_oo_df = outside_options_marks[outside_options_marks.apply(is_valid, axis=1)]
    
    # Collect codes of valid options into a list
    valid_codes = valid_oo_df['code'].tolist()
    
    # Append single grouped row
    grouped_valid_combos.append({
        'Initial_Modules': initial_modules,
        'Valid_Outside_Options': valid_codes,
        'Valid_Count': len(valid_codes)
    })

# Final grouped dataframe
valid_dataframe_grouped = pd.DataFrame(grouped_valid_combos)

# Show the result


In [241]:
valid_AC102_codes = valid_dataframe_grouped["Valid_Outside_Options"][0]
valid_outside_options_AC102 = outside_options_marks[outside_options_marks['code'].isin(valid_AC102_codes)]
valid_ST101_codes = valid_dataframe_grouped["Valid_Outside_Options"][1]
valid_outside_options_ST101 = outside_options_marks[outside_options_marks['code'].isin(valid_ST101_codes)]
valid_outside_options_ST101

,code,prerequisites,mutually_exclusive_courses,units,department,mean,q10,Median,q90,Pooled_SD,Total Takers,Years Counted,parsed_prereqs,parsed_exclusions
67,EC336,"['ST102', 'ST109', 'MA100']",[],0.5,Economics,81.200000,71.500,82.500000,89.5,8.300000,6.000000,1.0,"[ST102, ST109, MA100]",[]
38,DS105A,[],['DS105W'],0.5,Data Science Institute,73.800000,62.800,76.000000,84.0,9.700000,49.000000,1.0,[],[DS105W]
366,ST310,"['ST102', 'ST109']",['ST309'],0.5,Statistics,73.696471,65.425,74.500000,82.8,8.379447,63.750000,4.0,"[ST102, ST109]",[ST309]
367,ST311,"['ST102', 'ST109', 'ST101']",[],0.5,Statistics,72.214737,64.300,73.166667,81.1,9.171506,31.666667,3.0,"[ST102, ST109, ST101]",[]
78,EH215,[],['EH204'],0.5,Economic History,72.181818,66.900,71.500000,79.0,6.656325,5.500000,2.0,[],[EH204]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,PB102,[],[],1.0,Psychological and Behavioural Science,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
318,PH241,[],[],0.5,"Philosophy, Logic and Scientific Method",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
330,SP232,[],[],1.0,Social Policy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
331,SP314,[],[],0.5,Social Policy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]


In [264]:
valid_outside_options_ST101_clean = valid_outside_options_ST101.dropna(subset=['mean'])
valid_outside_options_AC102_clean = valid_outside_options_AC102 .dropna(subset=['mean'])

def print_sorted_by_mean(dataframe, num=5):
   
    # Step 1: Drop rows with NaN values in 'mean' column
    dataframe_clean = dataframe.dropna(subset=['mean'])

    # Step 2: Sort by 'mean' to identify the highest, lowest, and median mean
    sorted_df = dataframe_clean.sort_values(by='mean')

    # Step 3: Get the lowest, median, and highest rows
    lowest_index_start = 0
    lowest_index_end = num
    lowest_rows = sorted_df.iloc[lowest_index_start: lowest_index_end]

    median_index_start = len(sorted_df) // 2 - num // 2
    median_index_end = len(sorted_df) // 2 + num // 2 + (num % 2)
    median_rows = sorted_df.iloc[median_index_start: median_index_end]

    highest_index_start = len(sorted_df) - num
    highest_index_end = len(sorted_df)
    highest_rows = sorted_df.iloc[highest_index_start: highest_index_end]

    # Step 4: Output the results
    print(f"Lowest {num} mean rows:")
    print(lowest_rows[['code', 'mean', 'prerequisites', 'mutually_exclusive_courses','units']])

    print(f"\nMedian {num} mean rows:")
    print(median_rows[['code', 'mean', 'prerequisites', 'mutually_exclusive_courses','units']])

    print(f"\nHighest {num} mean rows:")
    print(highest_rows[['code', 'mean', 'prerequisites', 'mutually_exclusive_courses','units']])


* For ST101 the hardest modules ive chosen are MA102 in 2nd Year, and then LL106 in 3rd Year
          the median modules ive chosen are  AN237 in 2nd Year and then LL251 IN 3rd Year
          the easiest modules ive chosen are DS105A in 2nd Year and ST310 and ST311 in 3rd Year (what the department actually recommends in 3rd Year)
          
* For AC102 the hardest modules ive chosen are MA102 in 2nd Year, and then AC332 and EC339 in 3rd Year
          the median modules ive chosen are  AC342 in 2nd Year and then LL251 IN 3rd Year
          the easiest modules ive chosen are DS105A in 2nd Year and eh209 in 3rd Year 
          
* I have not picked the exact, easiest or hardest/median but rather courses which seem feasible for a student to choose and are similar to their initial option for added realism at a little cost to the effectiveness of the analysis


In [294]:

st101_modules_hard = ['ST101', 'MA102', 'LL106']  # Example of hard difficulty for ST101
st101_modules_medium = ['ST101', 'AN237', 'LL251']  # Medium difficulty
st101_modules_easy = ['ST101', 'DS105A', 'ST310', 'ST311']  # Easy difficulty

# AC102 Example (Replace with your actual choices)
ac102_modules_hard = ['AC102', 'MA102', 'AC332', 'EC339']  # Hard difficulty for AC102
ac102_modules_medium = ['AC102', 'AC342', 'LL251']  # Medium difficulty
ac102_modules_easy = ['AC102', 'DS105A', 'EH209']  # Easy difficulty



module_combinations = [
    {"Modules List": initial_combinations_df.iloc[1,0]+st101_modules_hard, 'Modules': st101_modules_hard, 'Difficulty': 'hard'},
    {"Modules List": initial_combinations_df.iloc[1,0]+st101_modules_medium,'Modules':st101_modules_medium, 'Difficulty': 'medium'},
    {"Modules List": initial_combinations_df.iloc[1,0]+st101_modules_easy, 'Modules':st101_modules_easy,'Difficulty': 'easy'},
    
    {"Modules List": initial_combinations_df.iloc[0,0]+ac102_modules_hard,'Modules':ac102_modules_hard, 'Difficulty': 'hard'},
    {"Modules List": initial_combinations_df.iloc[0,0]+ac102_modules_medium, 'Modules':ac102_modules_medium,'Difficulty': 'medium'},
    {"Modules List": initial_combinations_df.iloc[0,0]+ac102_modules_easy, 'Modules':ac102_modules_easy,'Difficulty': 'easy'},
]

# Create the new DataFrame from the list of dictionaries
manual_combinations_df = pd.DataFrame(module_combinations)
manual_combinations_df

,Modules List,Modules,Difficulty
0,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, ST101, ST101, MA102, LL106]","[ST101, MA102, LL106]",hard
1,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, ST101, ST101, AN237, LL251]","[ST101, AN237, LL251]",medium
2,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, ST101, ST101, DS105A, ST310, ST311]","[ST101, DS105A, ST310, ST311]",easy
3,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, AC102, AC102, MA102, AC332, EC339]","[AC102, MA102, AC332, EC339]",hard
4,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, AC102, AC102, AC342, LL251]","[AC102, AC342, LL251]",medium
5,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, AC102, AC102, DS105A, EH209]","[AC102, DS105A, EH209]",easy


looking at the average of average summary stats + lse classification of grades taken every grade at average


In [302]:
def calculate_statistics_for_modules(module_list, summary_df):
    # For each module, find the corresponding values (mean, q10, median, q90, Pooled_SD) in summary_df
    selected_modules = summary_df[summary_df['code'].isin(module_list)]

    # Check if any modules are found
    if not selected_modules.empty:
        # Calculate averages for each column
        mean_value = selected_modules['mean'].mean()
        q10_value = selected_modules['q10'].mean()
        median_value = selected_modules['Median'].mean()
        q90_value = selected_modules['q90'].mean()
        pooled_sd_value = selected_modules['Pooled_SD'].mean()
        
        return mean_value, q10_value, median_value, q90_value, pooled_sd_value
    else:
        return None, None, None, None, None

# Apply the function to calculate all statistics for each module list
manual_combinations_df[['Mean', 'Q10', 'Median', 'Q90', 'Pooled_SD']] = manual_combinations_df['Modules List'].apply(
    lambda x: pd.Series(calculate_statistics_for_modules(x, summary_df))
)
manual_combinations_df.to_csv("data/degrees/manual_combinations_df.csv", index=True)

# Display the updated dataframe with the calculated statistics
manual_combinations_df


,Modules List,Modules,Difficulty,Mean,Q10,Median,Q90,Pooled_SD
0,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, ST101, ST101, MA102, LL106]","[ST101, MA102, LL106]",hard,62.590013,47.113833,63.737083,77.539667,13.202202
1,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, ST101, ST101, AN237, LL251]","[ST101, AN237, LL251]",medium,63.635839,48.981333,64.708333,77.924417,12.547597
2,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, ST101, ST101, DS105A, ST310, ST311]","[ST101, DS105A, ST310, ST311]",easy,64.801075,50.151984,65.934921,79.131825,12.663701
3,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, AC102, AC102, MA102, AC332, EC339]","[AC102, MA102, AC332, EC339]",hard,62.648452,47.043413,63.953175,78.045159,13.269274
4,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, AC102, AC102, AC342, LL251]","[AC102, AC342, LL251]",medium,63.838501,49.248333,64.988333,78.230167,12.318626
5,"[MA108, FM100, FM102, EC1A3, EC1B3, ST102, FM201, FM200, EC2C3, EC2C4, EC2A3, FM215, FM214, FM301, FM302, FM321, FM322, FM304, FM305, AC102, AC102, DS105A, EH209]","[AC102, DS105A, EH209]",easy,64.493174,49.658083,65.657083,78.998667,12.603848


bit of statistical analysis: